In [2]:
#!/bin/python
import numpy as np
import keras.utils
import matplotlib.pyplot as plt
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
train = unpickle('test_batch')



In [3]:
def one_hot_representation(dlabel):
    col = len(dlabel)
    one_hot = np.zeros((10,col))
    k = 0
    for i in dlabel:
        one_hot[i][k] = 1
        k = k + 1
    return one_hot
def LoadBatch(train):
    trX = np.transpose(train[b'data'])
    trX = np.array(trX,dtype = float)
    trX = trX /255
    trY = keras.utils.to_categorical(train[b'labels'], num_classes=10)
    mean_x = np.mean(trX , axis=0)
    std_x = np.std(trX , axis=0)
    std_x = np.array(std_x)
    trX = (trX - mean_x)
    trX = trX/std_x
    trsy = train[b'labels']
    return trX,trY,trsy
'''
to try to build the whole dataset 
full_or_not : 
    1 if we want to use the whole batch data, 
    0 if we only take one batch as training and one batch as test
                                                                            '''
def Initialize_data(full_or_not):
    if full_or_not is 0 :
        # TRAINING
        train = unpickle('data_batch_1')
        (trx,trY,trsy) = LoadBatch(train)
        # VALIDATION
        val   = unpickle('data_batch_2')
        (valx,valY,valy) = LoadBatch(val)
        # TEST
        test  = unpickle('test_batch')
        (testx,testY,testy) = LoadBatch(test)
    else: 
        # TRAINING , val
        
        for i in range(5):
            name = 'data_batch_' + str(i+1)
            temp = unpickle(name)
            (tempx,tempY,tempy) = LoadBatch(temp)
            if i == 0 :
                trxt  = tempx
                trYt  = tempY
                trsyt = tempy
            else:
                trxt = np.concatenate((trxt,tempx),axis=1)
                trYt = np.concatenate((trYt,tempY),axis=0)
                trsyt = np.concatenate((trsyt,tempy),axis = 0)
    
        trx = trxt[:,0:48999]
        print(np.shape(trx)) 
        valx = trxt[:,49000:49999]
        print(np.shape(valx)) 
        trY = trYt[0:48999][:]
        valY = trYt[49000:49999][:]
        trsy = trsyt[0:48999]
        valy = trsyt[49000:49999]
        
        # test
        
        test  = unpickle('test_batch')
        (testx,testY,testy) = LoadBatch(test)
    return (trx.T,trY,trsy,valx.T,valY,valy,testx.T,testY,testy)
    
    
Initialize_data(1)

(3072, 48999)
(3072, 999)


(array([[-0.85518733, -1.16303523, -1.02835178, ...,  0.70329265,
         -0.37417499, -0.60506092],
        [ 0.39108766, -0.07188142, -0.41910822, ...,  0.14306851,
          0.19267234,  0.22574156],
        [ 1.46868548,  1.44446163,  1.44446163, ..., -0.61456596,
         -0.61456596, -0.60245403],
        ...,
        [ 0.15318951,  0.76665756,  0.8177799 , ..., -1.58496999,
         -1.63609233, -1.68721467],
        [-1.39391488, -1.72776589, -1.88356302, ...,  0.58693441,
          0.34211034, -0.1475378 ],
        [-1.23622319, -1.21788906, -1.05288185, ...,  2.44893787,
          2.39393547,  2.33893306]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32), array([6, 9, 9, ..., 9, 4, 9]), array([[ 0.4091438 ,  0.36649761,  0.34517452, ..., -0.10261045,
         -0.1

In [4]:
def category_names(dataset):
    if dataset == 'cifar10':
        return [
        'airplane',
        'automobile',
        'bird',
        'cat',
        'deer',
        'dog',
        'frog',
        'horse',
        'ship',
        'truck']
    else:
        raise ValueError("Unexpected dataset " + dataset + ".")

In [5]:
def ComputeAccuracy(predictions,testY):
    answers = predictions.argmax(axis=1)
    (d, ) = np.shape(testY)
    ii = np.reshape(np.array(testY),(d,1))
    y = np.reshape(np.array(answers),(d,1))		
    correct = ii - y
    correct[correct!=0] = -1
    correct[correct==0] = 1
    correct[correct==-1] = 0
    a = np.count_nonzero(correct)
    result = float(a)/float(d)
    return result

def Draw_loss(epochs,predictions,H):
    N = np.arange(0, epochs)
    plt.style.use("ggplot")
    #### print accuracy
    plt.plot(N, H.history["acc"], label="train_acc")
    plt.plot(N, H.history["val_acc"], label="val_acc")
    plt.title(" Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend()
    #### print loss
    plt.figure()
    plt.plot(N, H.history["loss"], label="train_loss")
    plt.plot(N, H.history["val_loss"], label="val_loss")
    plt.title("Training Loss ")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend()
   


    